In [101]:
# notebook copied from here:
# https://github.com/urchade/GLiNER/blob/main/examples/finetune.ipynb

import json
from gliner import GLiNER
import spacy
from gliner_spacy.pipeline import GlinerSpacy

import torch
from tqdm import tqdm
from transformers import get_cosine_schedule_with_warmup
import os

In [102]:
# this function loads jsonl data
def read_jsonl(file_path):
    with open(file_path, "r") as f:
        data = [json.loads(line) for line in f]
    return data

### Load NER Dataset for Fine-Tuning

**TODO**: you need to load your own NER dataset here

In [103]:
train_path = 'data/argilla_dataset_train.jsonl'
eval_path = 'data/argilla_dataset_eval.jsonl'
baseline_predictions_path = 'data/argilla_dataset_gliner_eval_preds.jsonl'

train_data = read_jsonl(train_path)
eval_data = read_jsonl(eval_path)
baseline_predictions = read_jsonl(baseline_predictions_path)
len(train_data), len(eval_data), len(baseline_predictions)

(969, 31, 31)

In [104]:
# checking for gliner format
# {'tokenized_text' [], 'ner': [ [start_token_i, end_token_i, label], ...], ...}
print(f"{train_data[0]}\n\n{eval_data[3]}")

{'tokenized_text': ['Dress', ',', 'Shoes', ',', 'and', 'Scarf', 'provided', 'by', 'ModCloth', '.'], 'ner': [[0, 5, 'clothing'], [7, 12, 'clothing'], [18, 23, 'clothing'], [36, 44, 'organization']]}

{'tokenized_text': ['You', 'can', 'bring', 'some', 'of', 'varying', 'weight', 'to', 'give', 'yourself', 'options', '.', 'Gaiters', '–', 'these', 'will', 'protect', 'you', 'and', 'your', 'boots', 'from', 'rain', 'or', 'muck', '.'], 'ner': [[105, 110, 'clothing'], [63, 70, 'organization']]}


### Load Pre-Trained Model

In [105]:
# available models: https://huggingface.co/urchade

model = GLiNER.from_pretrained("urchade/gliner_largev2")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/egemenipek/miniconda3/envs/zero-shot-ner-lab/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


### Fine-Tuning Code

In [106]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

import torch
from transformers import TrainerCallback
from gliner import GLiNERConfig, GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollatorWithPadding, DataCollator
from gliner.utils import load_config_as_namespace
from gliner.data_processing import WordsSplitter, GLiNERDataset

In [107]:
if torch.backends.mps.is_available():
    device = torch.device("mps") # if you have apple m-series 
elif torch.cuda.is_available():
    device = torch.device("cuda") # if you have gpu
else:
    device = torch.device("cpu") # you most likely have this :)

model = GLiNER.from_pretrained("urchade/gliner_largev2")

# dynamic padding used to speed training up and save memory
data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

model.to(device)
print(f"Model is on: {next(model.parameters()).device}")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Model is on: mps:0


In [108]:
num_epochs = 2  # Define how many times you want to pass over the dataset
batch_size = 16
data_size = len(train_data)
num_batches = data_size // batch_size  # Total batches per epoch
num_steps = num_epochs * num_batches  # Total training steps
print(f"Number of Epochs: {num_epochs}, Number of Batches: {num_batches}, Number of Steps: {num_steps}")

Number of Epochs: 2, Number of Batches: 60, Number of Steps: 120


In [109]:
training_args = TrainingArguments(
    output_dir="models",
    learning_rate=5e-6,
    weight_decay=0.01,
    others_lr=1e-5,
    others_weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    focal_loss_alpha=0.75,
    focal_loss_gamma=2,
    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    #save_steps = 100,
    #save_total_limit=10,
    dataloader_num_workers = 0,
    use_cpu = False,
    #report_to="none",
    )

# this is to track loss during training
class LossTrackerCallback(TrainerCallback):
    def __init__(self):
        self.losses = []  # Store loss per step

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            self.losses.append(logs["loss"])
            print(f"Step {state.global_step}: Loss {logs['loss']}")

loss_tracker = LossTrackerCallback()

/Users/egemenipek/miniconda3/envs/zero-shot-ner-lab/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [110]:
import tqdm
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
    callbacks=[loss_tracker]
)

trainer.train()

/var/folders/c_/w5by8m2s30jcjd665yxczwx40000gp/T/ipykernel_72830/3698432713.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Step,Training Loss,Validation Loss


TrainOutput(global_step=122, training_loss=9.800071841380635, metrics={'train_runtime': 153.0584, 'train_samples_per_second': 12.662, 'train_steps_per_second': 0.797, 'total_flos': 0.0, 'train_loss': 9.800071841380635, 'epoch': 2.0})

### Run Fine-Tuning

### Save and Load Fine-tuned Model

In [111]:
model.save_pretrained("models/fine-tuned-gliner")

In [116]:
fine_tuned_gliner = GLiNER.from_pretrained("models/fine-tuned-gliner", local_files_only=True)

config.json not found in /Users/egemenipek/zero-shot-ner-fine-tuning-lab/notebooks/models/fine-tuned-gliner


In [119]:
text = 'dress is a clotthing item. Look at that jacket yo!'

labels = ['clothing', 'organization', 'address', 'event']

entities = fine_tuned_gliner.predict_entities(text, labels)
print(entities)

[]


### Evaluate Model

In [ ]:
train_data[1]

In [ ]:
entity_types = sorted(set([t for item in eval_data for _, _, t in item['ner']]))
entity_types

In [ ]:
loaded_model.predict_entities("The Federal Reserve announced new guidelines for banking operations to enhance financial stability.", labels=['regulator'])

In [ ]:
eval_str, f1 = loaded_model.evaluate(test_data=eval_data, flat_ner=True, threshold=0.5, batch_size=1, entity_types=entity_types)
print(eval_str)

<h1> Evaluate and Compare

In [112]:
from nervaluate import Evaluator

# below function converts the data to the format that nervaluate expects
def convert_data_to_nervaluate_format(data):
    formatted_data = []
    for data_point in data:
        formatted_data_point = [{'label': ner_point[2], 'start': ner_point[0], 'end': ner_point[1]} for ner_point in data_point['ner']]
        formatted_data.append(formatted_data_point)
    return formatted_data

In [114]:
# predict evaluation set with the fine-tuned model

# fine-tuned model is the new GLiNER
fine_tuned_gliner = 'models/fine-tuned-gliner'

# use the same labels you have defined for this lab
zero_shot_labels = ['address', 'organization', 'person'] 

# Configuration for GLiNER integration
custom_spacy_config = {
    "gliner_model": fine_tuned_gliner,
    "chunk_size": 250,
    "labels": zero_shot_labels,
    "style": "ent"
}

# Initialize a blank English spaCy pipeline and add GLiNER
nlp = spacy.blank("en")
nlp.add_pipe("gliner_spacy", config=custom_spacy_config)

text = "clothes clothing dress tshirt"

# Process the text with the pipeline
doc = nlp(text)

# Output detected entities
for ent in doc.ents:
    print(ent.text, ent.label_)

config.json not found in /Users/egemenipek/zero-shot-ner-fine-tuning-lab/notebooks/models/fine-tuned-gliner
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


address address organization organization


In [ ]:
true = convert_data_to_nervaluate_format(eval_data)
pred = convert_data_to_nervaluate_format()
baseline_pred = convert_data_to_nervaluate_format(baseline_predictions)

In [ ]:
true = convert_data_to_nervaluate_format(eval_set) # human annotated data in the format that nervaluate expects
pred = convert_data_to_nervaluate_format(gliner_eval_preds) # GLiNER predictions in the format that nervaluate expects

evaluator = Evaluator(true, pred, tags=labels)
results, results_per_tag, result_indices, result_indices_by_tag = evaluator.evaluate()

print(f"Precision: {results['ent_type']['precision']}\nRecall: {results['ent_type']['recall']}\nF1: {results['ent_type']['f1']}")

### Evaluation with Nervaluate

In [ ]:
eval_predictions = []
for item in eval_data:
    text = ' '.join(item["tokenized_text"])
    predictions = loaded_model.predict_entities(text, labels=entity_types)
    eval_predictions.append(predictions)

In [ ]:
def format_data_for_nervaluate(data):
    """
    Example for required format:
    true = [
    [{"label": "PER", "start": 2, "end": 4}],
    [{"label": "LOC", "start": 1, "end": 2},
     {"label": "LOC", "start": 3, "end": 4}]
    ]
    pred = [
        [{"label": "PER", "start": 2, "end": 4}],
        [{"label": "LOC", "start": 1, "end": 2},
        {"label": "LOC", "start": 3, "end": 4}]
    ]
    """
    formatted_data = []
    for item in data:
        formatted_entities = []
        for start, end, label in item['ner']:
            formatted_entities.append({"label": label, "start": start, "end": end})
        formatted_data.append(formatted_entities)
    return formatted_data

In [ ]:
formatted_eval_data = format_data_for_nervaluate(eval_data)

In [ ]:
from nervaluate import Evaluator
from pprint import pprint

# true = [
#     [{"label": "PER", "start": 2, "end": 4}],
#     [{"label": "LOC", "start": 1, "end": 2},
#      {"label": "LOC", "start": 3, "end": 4}]
# ]

# pred = [
#     [{"label": "PER", "start": 2, "end": 4}],
#     [{"label": "LOC", "start": 1, "end": 2},
#      {"label": "LOC", "start": 3, "end": 4}]
# ]

# evaluator = Evaluator(true, pred, tags=['LOC', 'PER'])

evaluator = Evaluator(formatted_eval_data, eval_predictions, tags=['tool'])

# Returns overall metrics and metrics for each tag

results, results_per_tag = evaluator.evaluate()

pprint(results)

[{'tokenized_text': ['I', 'was', 'searching', 'for', 'a', 'Property', 'and', 'found', 'this', 'listing', '(', 'MLS', '®', '#', '10100074', ')', '.', 'Please', 'send', 'me', 'more', 'information', 'regarding', '16440', 'Cr', '178', '#', '1001', ',', 'Tyler', ',', 'TX', ',', '75703', '.'], 'ner': [[114, 150, 'address']]}, {'tokenized_text': ['Family', 'flat', 'located', 'on', 'an', 'upper', 'floor', '(', 'with', 'lift', ')', 'in', 'the', 'North', 'historical', 'Marais', 'area', '-', 'right', 'bank', 'of', 'Paris', '.'], 'ner': []}, {'tokenized_text': ['|a', 'Kerkham', ',', 'Roger', '.', '|0', 'http://id', '.', 'loc', '.', 'gov', '/', 'authorities', '/', 'names', '/', 'n50044780', '.'], 'ner': [[3, 17, 'person']]}, {'tokenized_text': ['.', '.', 'nice', 'and', 'airy', 'condo', '.', '.', '.', 'with', 'lovely', 'home', '-', 'cooked', 'dishes', 'by', 'Sugen', "'s", 'grandma', '!', 'All', 'from', 'scratch', '!', 'Such', 'a', 'talented', 'lady', 'who', 'can', 'also', 'crochet', 'and', 'cross', 